# Advanced Pandas

In [1]:
import pandas as pd
import numpy as np

## Categorical Data

### Background and Motivation

In [2]:
values = pd.Series(['apple', 'orange', 'apple',
                     'apple'] * 2)

In [3]:
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [4]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [5]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [6]:
values = pd.Series([0, 1, 0, 0] * 2)

In [7]:
dim = pd.Series(['apple', 'orange'])

In [8]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [9]:
dim

0     apple
1    orange
dtype: object

In [10]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### Categorical Type in pandas

In [11]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2

In [12]:
N= len(fruits)

In [13]:
N

8

In [14]:
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])

In [15]:
df

,basket_id,fruit,count,weight
0,0,apple,8,2.768654
1,1,orange,12,1.535508
2,2,apple,11,1.577557
3,3,apple,7,3.649062
4,4,apple,6,0.188839
5,5,orange,11,1.231936
6,6,apple,9,3.155890
7,7,apple,5,2.812201


In [16]:
fruit_cat = df['fruit'].astype('category')

In [17]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [18]:
c = fruit_cat.values

In [19]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [20]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [21]:
df['fruit'] = df['fruit'].astype('category')

In [22]:
df['fruit'] 

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [23]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

In [24]:
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [25]:
categories= ['foo', 'bar', 'baz']

In [26]:
codes = [0,1,2,0,1,2]

In [27]:
my_cat_2 = pd.Categorical.from_codes(codes, categories)

In [28]:
my_cat_2

['foo', 'bar', 'baz', 'foo', 'bar', 'baz']
Categories (3, object): ['foo', 'bar', 'baz']

In [29]:
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                        ordered=True)

In [30]:
ordered_cat

['foo', 'bar', 'baz', 'foo', 'bar', 'baz']
Categories (3, object): ['foo' < 'bar' < 'baz']

### Computations with Categoricals

In [31]:
np.random.seed(12345)

In [32]:
draws = np.random.randn(1000)

In [33]:
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [34]:
# qcut()是按变量的数量来对变量进行分割，并且尽量保证每个分组里变量的个数相同。
bins = pd.qcut(draws,4)

In [35]:
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [36]:
bins = pd.qcut(draws, 4 , labels=['Q1', 'Q2', 'Q3', 'Q4'])

In [37]:
bins

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [38]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [39]:
bins = pd.Series(bins, name='quartile')

In [40]:
result = (pd.Series(draws).groupby(bins).agg(['count','min','max']).reset_index())

In [41]:
result

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [42]:
result['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

#### Better performance with categoricals

In [43]:
N = 10000000

In [44]:
draws = pd.Series(np.random.randn(N))

In [45]:
labels = pd.Series(['foo','bar','baz','qux'] *(N//4))

In [46]:
labels

0          foo
1          bar
2          baz
3          qux
4          foo
          ... 
9999995    qux
9999996    foo
9999997    bar
9999998    baz
9999999    qux
Length: 10000000, dtype: object

In [47]:
categories = labels.astype('category')

In [48]:
labels.memory_usage()

80000128

In [50]:
 categories.memory_usage()

10000320

### Categorical Methods

In [51]:
s = pd.Series(['a','b','c','d'] * 2)

In [52]:
cat_s = s.astype('category')

In [53]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [54]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [55]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [59]:
actual_categories = ['a', 'b', 'c', 'd', 'e']

In [60]:
cat_s2 = cat_s.cat .set_categories(actual_categories)

In [61]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [64]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]

In [65]:
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [67]:
cat_s2.cat.remove_unused_categories()

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

#### Creating dummy variables for modeling

In [68]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [69]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## Advanced GroupBy Use

In [71]:
 df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                     'value': np.arange(12.)})

In [72]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


## Group Transforms and "Unwrapped" GroupBys

In [73]:
g = df.groupby('key').value

In [74]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [75]:
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [76]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [84]:
g.apply(lambda x : x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [78]:
g.transform (lambda x : x.rank(ascending = False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [81]:
def normalize(x) :
    return (x- x.mean()) / x.std()

In [82]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [83]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### Grouped Time Resampling

In [85]:
N =15

In [86]:
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)

In [87]:
df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})

In [88]:
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [89]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [90]:
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})

In [91]:
df2

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0
7,2017-05-20 00:02:00,b,7.0
8,2017-05-20 00:02:00,c,8.0
9,2017-05-20 00:03:00,a,9.0


In [105]:
time_key = pd.Grouper('5min')

In [106]:
time_key.

Grouper(key='5min', axis=0, sort=False)